In [1]:
import pandas as pd
import numpy as np
import datetime as DT
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
import random
from sklearn import tree

In [2]:
# Solo me voy a quedar con las vistas de los usuarios

In [3]:
vistas = pd.concat([
    pd.read_csv("../input/datos/fiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_3_vistas.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_3_vistas.csv")
])

In [4]:
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)
vistas.head()

,idaviso,timestamp,idpostulante
0,1111780242,2018-02-23T13:38:13.187-0500,YjVJQ6Z
1,1112263876,2018-02-23T13:38:14.296-0500,BmVpYoR
2,1112327963,2018-02-23T13:38:14.329-0500,wVkBzZd
3,1112318643,2018-02-23T13:38:17.921-0500,OqmP9pv
4,1111903673,2018-02-23T13:38:18.973-0500,DrpbXDP


In [5]:
vistas = vistas[['idaviso', 'idpostulante']]
vistas.drop_duplicates(subset=['idaviso', 'idpostulante'], inplace=True)
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [6]:
# Cargo los avisos
avisos = pd.concat([
    pd.read_csv("../input/datos/fiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/DESfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/ENTfiuba_6_avisos_detalle.csv"),
    pd.read_csv("../input/datosdatos/MISfiuba_6_avisos_detalle_missing_nivel_laboral.csv")
])

In [7]:
avisos.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [8]:
# Elimino las columnas que no me interesan
avisos = avisos[['idaviso', 'nombre_area']]
avisos.head()

,idaviso,nombre_area
0,8725750,Comercial
1,17903700,Salud
2,1000150677,Transporte
3,1000610287,Transporte
4,1000872556,Producción


In [9]:
avisos.drop_duplicates(subset=['idaviso', 'nombre_area'], inplace=True)

In [10]:
areas_importantes = [
    'Ventas', 'Administración', 'Producción', 'Comercial', 'Atención al Cliente',
    'Recepcionista', 'Call Center', 'Telemarketing', 'Tesorería', 'Mantenimient y Limpieza',
]

In [11]:
avisos['nombre_area'] = avisos['nombre_area'].apply(lambda x: x if x in areas_importantes else 'Otra Area')

In [12]:
avisos.head()

,idaviso,nombre_area
0,8725750,Comercial
1,17903700,Otra Area
2,1000150677,Otra Area
3,1000610287,Otra Area
4,1000872556,Producción


In [13]:
# Uno avisos y vistas.
avisos_vistos = pd.merge(vistas, avisos, on='idaviso', how='left')
avisos_vistos.head()

,idaviso,idpostulante,nombre_area
0,1111780242,YjVJQ6Z,Otra Area
1,1112263876,BmVpYoR,Otra Area
2,1112327963,wVkBzZd,Otra Area
3,1112318643,OqmP9pv,Administración
4,1111903673,DrpbXDP,Otra Area


In [14]:
avisos_vistos['n'] = 1

In [15]:
avisos_vistos['nombre_area'].value_counts()

Otra Area              2492285
Ventas                  763164
Administración          532516
Comercial               378112
Producción              362817
Atención al Cliente     327444
Recepcionista           263715
Call Center             204285
Tesorería               149429
Telemarketing           117380
Name: nombre_area, dtype: int64

In [16]:
vistos_por_area = avisos_vistos.pivot_table(index='idpostulante', values='n', columns='nombre_area', aggfunc='count')

In [17]:
vistos_por_area.fillna(0, inplace=True)

In [18]:
vistos_por_area.reset_index(inplace=True)

In [19]:
vistos_por_area.head()

nombre_area,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0005E,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,00Lkv,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,00dMd,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,01QAq,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,021OM,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# POSTULACIONES
postulaciones = pd.concat([
    pd.read_csv('../input/datos/fiuba_4_postulaciones.csv'),
    pd.read_csv('../input/datosdatos/ENTfiuba_4_postulaciones.csv'),
])

In [21]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [22]:
postulaciones = postulaciones[['idaviso', 'idpostulante']]
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [23]:
postulaciones.drop_duplicates(subset=['idaviso','idpostulante'], inplace=True)
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [24]:
# obtengo el detalle de cada aviso postulado
avisos_postulados = pd.merge(postulaciones, avisos, on='idaviso', how='left')

In [25]:
avisos_postulados.head()

,idaviso,idpostulante,nombre_area
0,1112257047,NM5M,Atención al Cliente
1,1111920714,NM5M,Telemarketing
2,1112346945,NM5M,Telemarketing
3,1112345547,NM5M,Telemarketing
4,1112237522,5awk,Otra Area


In [26]:
#UNO AMBOS

In [27]:
datos = pd.merge(vistos_por_area, avisos_postulados, on='idpostulante', how='inner')
datos.head()

,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,idaviso,nombre_area
0,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112384041,Otra Area
1,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112420060,Otra Area
2,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112257171,Otra Area
3,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112281879,Comercial
4,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112292169,Otra Area


In [28]:
# 1 POSTULA
# 0 no postula

In [29]:
datos['clase'] = 1

In [30]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5452123 entries, 0 to 5452122
Data columns (total 14 columns):
idpostulante           object
Administración         float64
Atención al Cliente    float64
Call Center            float64
Comercial              float64
Otra Area              float64
Producción             float64
Recepcionista          float64
Telemarketing          float64
Tesorería              float64
Ventas                 float64
idaviso                int64
nombre_area            object
clase                  int64
dtypes: float64(10), int64(2), object(2)
memory usage: 623.9+ MB


In [31]:
# CREO NO_POSTULACIONES

In [32]:
avisos.head()

,idaviso,nombre_area
0,8725750,Comercial
1,17903700,Otra Area
2,1000150677,Otra Area
3,1000610287,Otra Area
4,1000872556,Producción


In [33]:
avisos_random = avisos.sample(500000, replace=True)

In [34]:
avisos_random.head()

,idaviso,nombre_area
4306,1112296964,Otra Area
13631,1112404200,Otra Area
11084,1111944757,Otra Area
3958,1112191370,Otra Area
8356,1112484846,Otra Area


In [35]:
avisos_random.reset_index(inplace=True)
avisos_random.head()

,index,idaviso,nombre_area
0,4306,1112296964,Otra Area
1,13631,1112404200,Otra Area
2,11084,1111944757,Otra Area
3,3958,1112191370,Otra Area
4,8356,1112484846,Otra Area


In [36]:
avisos_random.drop(columns='index', inplace=True)
avisos_random.head()

,idaviso,nombre_area
0,1112296964,Otra Area
1,1112404200,Otra Area
2,1111944757,Otra Area
3,1112191370,Otra Area
4,1112484846,Otra Area


In [37]:
vistos_por_area.head()

nombre_area,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0005E,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,00Lkv,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,00dMd,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,01QAq,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,021OM,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [38]:
vistos_por_area_random = vistos_por_area.sample(500000, replace=True)

In [39]:
vistos_por_area_random = vistos_por_area_random.reset_index()

In [40]:
vistos_por_area_random.drop(columns='index', inplace=True)

In [41]:
vistos_por_area_random.head()

nombre_area,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,mYwXzQ,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Eze8lRo,0.0,2.0,0.0,1.0,6.0,1.0,0.0,0.0,1.0,4.0
2,A36O2ej,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
3,EzZ80bz,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EkKbq4,3.0,2.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,1.0


In [42]:
# los juntos
datos_no_postulados = pd.concat([vistos_por_area_random, avisos_random], axis=1)

In [43]:
datos_no_postulados.head()

,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,idaviso,nombre_area
0,mYwXzQ,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1112296964,Otra Area
1,Eze8lRo,0.0,2.0,0.0,1.0,6.0,1.0,0.0,0.0,1.0,4.0,1112404200,Otra Area
2,A36O2ej,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1111944757,Otra Area
3,EzZ80bz,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1112191370,Otra Area
4,EkKbq4,3.0,2.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,1.0,1112484846,Otra Area


In [44]:
# 0 no postulado
datos_no_postulados['clase'] = 0

In [45]:
# Finalmente uno ambos
datos_ml = pd.concat([datos, datos_no_postulados], axis=0)

In [46]:
datos_ml.head()

,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,idaviso,nombre_area,clase
0,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112384041,Otra Area,1
1,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112420060,Otra Area,1
2,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112257171,Otra Area,1
3,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112281879,Comercial,1
4,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112292169,Otra Area,1


In [47]:
# Elimino duplicados, pero quedandome con los POSTULADOS
datos_ml.drop_duplicates(subset=['idpostulante', 'idaviso'], inplace=True)

In [48]:
datos_ml.head()

,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas,idaviso,nombre_area,clase
0,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112384041,Otra Area,1
1,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112420060,Otra Area,1
2,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112257171,Otra Area,1
3,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112281879,Comercial,1
4,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112292169,Otra Area,1


In [49]:
# ULTIMO: Decision Tree no acepta categoricos.
dummies_area = pd.get_dummies(datos_ml['nombre_area'])

In [50]:
datos_ml.columns = ['idpostulante', 'vistas Administración', 'vistas Atención al Cliente',
                    'vistas Call Center', 'vistas Comercial', 'vistas Otra Area', 
                    'vistas Producción', 'vistas Recepcionista', ' vistas Telemarketing',
                    'vistas Tesorería', 'vistas Ventas',  'idaviso', 'nombre_area',
                    'clase']
datos_ml.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,nombre_area,clase
0,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112384041,Otra Area,1
1,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112420060,Otra Area,1
2,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112257171,Otra Area,1
3,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112281879,Comercial,1
4,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112292169,Otra Area,1


In [51]:
datos_ml = pd.concat([datos_ml, dummies_area], axis=1)
datos_ml.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,nombre_area,clase,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112384041,Otra Area,1,0,0,0,0,1,0,0,0,0,0
1,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112420060,Otra Area,1,0,0,0,0,1,0,0,0,0,0
2,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112257171,Otra Area,1,0,0,0,0,1,0,0,0,0,0
3,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112281879,Comercial,1,0,0,0,1,0,0,0,0,0,0
4,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112292169,Otra Area,1,0,0,0,0,1,0,0,0,0,0


In [52]:
datos_ml.drop(columns='nombre_area', inplace=True)
datos_ml.head()

,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,idaviso,clase,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112384041,1,0,0,0,0,1,0,0,0,0,0
1,0z5Dmrd,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1112420060,1,0,0,0,0,1,0,0,0,0,0
2,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112257171,1,0,0,0,0,1,0,0,0,0,0
3,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112281879,1,0,0,0,1,0,0,0,0,0,0
4,0z5JW1r,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,1112292169,1,0,0,0,0,1,0,0,0,0,0


In [53]:
# CARGO LOS DATOS 100k y lo llevo a un formato de ML

In [54]:
final100k = pd.read_csv('../input/finalfinal/test_final_100k.csv')
final100k.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [55]:
#merge1 = pd.merge(final100k,)
m1 = pd.merge(final100k, vistos_por_area, on='idpostulante', how='left')
m1.head()

,id,idaviso,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0,739260,6M9ZQR,0.0,0.0,0.0,3.0,14.0,3.0,0.0,0.0,0.0,2.0
1,1,739260,6v1xdL,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,2,739260,ezRKm9,0.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0
3,3,758580,1Q35ej,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,758580,EAN4J6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [56]:
m1.fillna(0, inplace=True)

In [57]:
m1.head()

,id,idaviso,idpostulante,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0,739260,6M9ZQR,0.0,0.0,0.0,3.0,14.0,3.0,0.0,0.0,0.0,2.0
1,1,739260,6v1xdL,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,2,739260,ezRKm9,0.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0
3,3,758580,1Q35ej,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [58]:
m1.columns=['id','idaviso','idpostulante', 'vistas Administración', 'vistas Atención al Cliente',
                    'vistas Call Center', 'vistas Comercial', 'vistas Otra Area', 
                    'vistas Producción', 'vistas Recepcionista', ' vistas Telemarketing',
                    'vistas Tesorería', 'vistas Ventas']
m1.head()

,id,idaviso,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas
0,0,739260,6M9ZQR,0.0,0.0,0.0,3.0,14.0,3.0,0.0,0.0,0.0,2.0
1,1,739260,6v1xdL,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,2,739260,ezRKm9,0.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0
3,3,758580,1Q35ej,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,758580,EAN4J6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
avisos.head()

,idaviso,nombre_area
0,8725750,Comercial
1,17903700,Otra Area
2,1000150677,Otra Area
3,1000610287,Otra Area
4,1000872556,Producción


In [60]:
# Nuevamente genero dummies
dummies_area = pd.get_dummies(avisos['nombre_area'])

In [61]:
avisos = pd.concat([avisos, dummies_area], axis=1)

In [62]:
avisos.head()

,idaviso,nombre_area,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,8725750,Comercial,0,0,0,1,0,0,0,0,0,0
1,17903700,Otra Area,0,0,0,0,1,0,0,0,0,0
2,1000150677,Otra Area,0,0,0,0,1,0,0,0,0,0
3,1000610287,Otra Area,0,0,0,0,1,0,0,0,0,0
4,1000872556,Producción,0,0,0,0,0,1,0,0,0,0


In [63]:
avisos.drop(columns='nombre_area', inplace=True)

In [64]:
avisos.head()

,idaviso,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,8725750,0,0,0,1,0,0,0,0,0,0
1,17903700,0,0,0,0,1,0,0,0,0,0
2,1000150677,0,0,0,0,1,0,0,0,0,0
3,1000610287,0,0,0,0,1,0,0,0,0,0
4,1000872556,0,0,0,0,0,1,0,0,0,0


In [65]:
m2 = pd.merge(m1, avisos, on='idaviso', how='left')

In [66]:
m2.head()

,id,idaviso,idpostulante,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0,739260,6M9ZQR,0.0,0.0,0.0,3.0,14.0,3.0,0.0,0.0,0.0,2.0,0,0,0,1,0,0,0,0,0,0
1,1,739260,6v1xdL,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0,0,1,0,0,0,0,0,0
2,2,739260,ezRKm9,0.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,1.0,2.0,0,0,0,1,0,0,0,0,0,0
3,3,758580,1Q35ej,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0
4,4,758580,EAN4J6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,1,0,0,0,0,0


In [67]:
X_final = m2.drop(columns=['id', 'idaviso','idpostulante'])

**ARBOLES DE DECISION**
.

In [68]:
y = datos_ml['clase']
X = datos_ml.drop(columns=['clase', 'idpostulante', 'idaviso'])

In [69]:
X.head()

,vistas Administración,vistas Atención al Cliente,vistas Call Center,vistas Comercial,vistas Otra Area,vistas Producción,vistas Recepcionista,vistas Telemarketing,vistas Tesorería,vistas Ventas,Administración,Atención al Cliente,Call Center,Comercial,Otra Area,Producción,Recepcionista,Telemarketing,Tesorería,Ventas
0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0
1,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0
2,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,0,0,0,0,1,0,0,0,0,0
3,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,0,0,0,1,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,2.0,0,0,0,0,1,0,0,0,0,0


In [70]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: clase, dtype: int64

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
for i in range(1,10): 
    clf = tree.DecisionTreeClassifier(criterion = "gini", random_state = 100,
                               max_depth=i, min_samples_leaf=i+5)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)  
    print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

In [73]:
clf = clf.fit(X_train, y_train)

In [74]:
y_pred = clf.predict(X_test)  

In [75]:
print('accuracy: {0:.2f}%'.format(accuracy_score(y_test,y_pred)*100))

accuracy: 91.58%


In [76]:
y_pred_final = clf.predict(X_final)

In [77]:
y_final = pd.DataFrame(y_pred_final)

In [78]:
# reconstruyo los datos

In [79]:
loquesea = pd.concat([m2['id'], y_final], axis=1)

In [80]:
loquesea.drop_duplicates(subset='id', inplace=True)

In [81]:
loquesea.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 100914
Data columns (total 2 columns):
id    100000 non-null int64
0     100000 non-null int64
dtypes: int64(2)
memory usage: 2.3 MB


In [82]:
loquesea.to_csv("summit001.csv", index=False)